In [1]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
import os
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [2]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


In [3]:
client = mlflow.tracking.MlflowClient()

In [4]:
myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

experiment_name = 'demo'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
model_name = 'demo'+'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name
print(experiment_name)

demo-bryandomino-ML_Automation


In [5]:
experiment = client.get_experiment_by_name(name=experiment_name)
#if(experiment is None):
#    print('Creating experiment ')
#    client.create_experiment(name=experiment_name)
#    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

2024/12/11 10:36:50 INFO mlflow.tracking.fluent: Experiment with name 'demo-bryandomino-ML_Automation' does not exist. Creating a new experiment.


demo-bryandomino-ML_Automation


<Experiment: artifact_location='mlflow-artifacts:/mlflow', creation_time=1733913410678, experiment_id='24', last_update_time=1733913410678, lifecycle_stage='active', name='demo-bryandomino-ML_Automation', tags={'mlflow.domino.dataset_info': '6758753d011f5130a7ff44db-6758753d011f5130a7ff44da',
 'mlflow.domino.environment_id': '673252a4f5f7702732bb4d82',
 'mlflow.domino.environment_revision_id': '673252a4f5f7702732bb4d84',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '6758753a011f5130a7ff44d5',
 'mlflow.domino.project_name': 'ML_Automation',
 'mlflow.domino.run_id': '67596ad2011f5130a7ff4721',
 'mlflow.domino.run_number': '2',
 'mlflow.domino.user': 'bryandomino',
 'mlflow.domino.user_id': '67346ea983fb127778bb7045',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'bryandomino'}>

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [7]:
# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
while(alpha<1):
    with mlflow.start_run():
        print('Start Run')
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(mlflow.get_tracking_uri())
        print(tracking_url_type_store)
        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
            mlflow.sklearn.log_model(lr, "demo_model", registered_model_name=model_name)

        else:        
            mlflow.sklearn.log_model(lr,  model_name)
        print(mlflow.get_artifact_uri())
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05        
        

Start Run
Alpha : 0.7
L1_Ratio : 0.6
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.7611277983559506
  MAE: 0.6320973951806732
  R2: 0.05393739148937782
http://127.0.0.1:8765
http


2024/12/11 10:37:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'demo-ML_Automation'.
2024/12/11 10:37:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-ML_Automation, version 1
Created version '1' of model 'demo-ML_Automation'.
2024/12/11 10:37:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-pig-953 at: http://127.0.0.1:8765/#/experiments/24/runs/8d165bb30d3545a2a130f2060105210e.
2024/12/11 10:37:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8765/#/experiments/24.


mlflow-artifacts:/mlflow/8d165bb30d3545a2a130f2060105210e/artifacts
Start Run
Alpha : 0.7999999999999999
L1_Ratio : 0.65
Elasticnet model (alpha=0.800000, l1_ratio=0.650000):
  RMSE: 0.7617841514999848
  MAE: 0.6334033261192783
  R2: 0.052305027239109125
http://127.0.0.1:8765
http


2024/12/11 10:37:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'demo-ML_Automation' already exists. Creating a new version of this model...
2024/12/11 10:37:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-ML_Automation, version 2
Created version '2' of model 'demo-ML_Automation'.
2024/12/11 10:37:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-hog-637 at: http://127.0.0.1:8765/#/experiments/24/runs/80a0788802654e6f8777c9a6a26c249e.
2024/12/11 10:37:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8765/#/experiments/24.


mlflow-artifacts:/mlflow/80a0788802654e6f8777c9a6a26c249e/artifacts
Start Run
Alpha : 0.8999999999999999
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.900000, l1_ratio=0.700000):
  RMSE: 0.7621458207483331
  MAE: 0.6344618878521748
  R2: 0.051404946838846444
http://127.0.0.1:8765
http


2024/12/11 10:37:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'demo-ML_Automation' already exists. Creating a new version of this model...
2024/12/11 10:37:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-ML_Automation, version 3
Created version '3' of model 'demo-ML_Automation'.
2024/12/11 10:37:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run luminous-loon-431 at: http://127.0.0.1:8765/#/experiments/24/runs/b8490fbbb5084345a21c56e0d4fed862.
2024/12/11 10:37:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8765/#/experiments/24.


mlflow-artifacts:/mlflow/b8490fbbb5084345a21c56e0d4fed862/artifacts
Start Run
Alpha : 0.9999999999999999
L1_Ratio : 0.7500000000000001
Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.7625575567637911
  MAE: 0.6356164107998428
  R2: 0.050379746053483276
http://127.0.0.1:8765
http


2024/12/11 10:37:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'demo-ML_Automation' already exists. Creating a new version of this model...
2024/12/11 10:37:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo-ML_Automation, version 4
Created version '4' of model 'demo-ML_Automation'.
2024/12/11 10:37:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-finch-854 at: http://127.0.0.1:8765/#/experiments/24/runs/b2a232ac6d86410eb3e428f650daead9.
2024/12/11 10:37:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8765/#/experiments/24.


mlflow-artifacts:/mlflow/b2a232ac6d86410eb3e428f650daead9/artifacts
